In [1]:
import recordlinkage
import pandas as pd
import re 
import numpy as np
import sys
from recordlinkage.base import BaseIndexAlgorithm
from recordlinkage.base import BaseCompareFeature
sys.path.append("..")
from modules.SoundexBlocking import SoundexBlocking
import re 
import Levenshtein as lev

# Cleaning

In [2]:
# Cleaning
STRING_COLUMNS = ['title','artist','track01']

# replace NaN with '' on STRING_COLUMNS
def clean_strings(df):
  df[STRING_COLUMNS] = df[STRING_COLUMNS].replace(np.nan, '')
  return df

# We don't remove non alphanumerics now, because we wouldn't be able to 
# undestand anything when we analyze the data. So we call this function 
# when we calculate the strings_distance
def remove_non_alphanum(string: str):
  return re.sub(r'\W+', '', string)

UNUSED_COLUMNS = ['id',	"category","genre","cdextra","year","track02","track03","track04","track05","track06","track07","track08","track09","track10","track11","track12","track13","track14","track15","track16","track17","track18","track19","track20","track21","track22","track23","track24","track25","track26","track27","track28","track29","track30","track31","track32","track33","track34","track35","track36","track37","track38","track39","track40","track41","track42","track43","track44","track45","track46","track47","track48","track49","track50","track51","track52","track53","track54","track55","track56","track57","track58","track59","track60","track61","track62","track63","track64","track65","track66","track67","track68","track69","track70","track71","track72","track73","track74","track75","track76","track77","track78","track79","track80","track81","track82","track83","track84","track85","track86","track87","track88","track89","track90","track91","track92","track93","track94","track95","track96","track97","track98","track99"]
def remove_unused_columns(df):
  return df.drop(columns = UNUSED_COLUMNS)

# a single function that calls all the above clean functions
def clean_db(df):
  df = remove_unused_columns(df) # TODO : try not cleaning (much data in memory) to see how to improve operations
  df = clean_strings(df)
  return df

# Ded

In [3]:
df = pd.read_csv('cd.csv', delimiter=';', doublequote=False)
df = clean_db(df)
df

,pk,artist,title,track01
0,1818,pink floyd,the wall disc 1,in the flesh
1,1823,Carlos Santana,Carlos,(Da Le) Yaleo
2,1828,Frans Bauer,'n ons geluk,'n ONS geluk
3,1838,2Pac,Greatest Hits (CD1),Keep Ya Head Up
4,1858,Ì¤ÃÎ,Ì¤ÃÎ,¥È¥é¥Ã¥¯ 1
...,...,...,...,...
9758,109759,Various,Warriors of Virtue,You Can Fly
9759,109760,James Patterson,Pop Goes the Weasel - CD 4,4.01
9760,109761,Colin Baker & Nicholas Courtney,The Spectre of Lanyon Moor,Episode 0
9761,109762,Studio Cutz,Volume 25,All Access 3:12


### Blocking

In [4]:
class FirstLetterWIndex(BaseIndexAlgorithm):
    """Custom class for indexing"""

    def _link_index(self, df_a, df_b):
        """Make pairs with given names starting with the letter 'w'."""
        blocker = SoundexBlocking('title')
        
        new_blocks = blocker.generate_blocks(df_a)   

        return new_blocks

        # # Select records with names starting with a w.
        # name_a_w = df_a[df_a['given_name'].str.startswith('w') == True]
        # name_b_w = df_b[df_b['given_name'].str.startswith('w') == True]

        # # Make a product of the two numpy arrays
        # return pd.MultiIndex.from_product(
        #     [name_a_w.index.values, name_b_w.index.values],
        #     names=[df_a.index.name, df_b.index.name]
        # )

In [5]:
indexer = recordlinkage.Index()
indexer.block("title")
candidate_links = indexer.index(df)
len(candidate_links)

703

### Comparer

In [6]:
class CompareStrings(BaseCompareFeature):
    def remove_non_alphanum(string: str):
        return re.sub(r'\W+', '', string)
    
    def string_distance(self, string1: str, string2: str):
        x = remove_non_alphanum(string1).lower()
        y = remove_non_alphanum(string2).lower()
        
        dist = lev.distance(x, y) 

        max_len = max(len(x), len(y))
        if (max_len == 0):
            return 0
        normalized = (max_len-dist) / max_len 
        normalized_dist = 1-normalized
        return normalized_dist
    
    def _compute_vectorized(self, s1, s2):
        print(s1)
        print(s2)
        print('_________\n\n')
        return 1

In [11]:
compare_cl = recordlinkage.Compare()
# compare_cl.add(CompareStrings("title", "title"))
compare_cl.string("title", "title", method="levenshtein", threshold=.85)
compare_cl.string("artist", "artist", method="levenshtein", threshold=.85)
features = compare_cl.compute(candidate_links, df)

# matches = features[features.sum(axis=1) > 1]
# matches
features.iloc[0: 5]

,,0,1
410,4,1.0,1.0
18,11,1.0,1.0
106,13,1.0,1.0
235,17,1.0,1.0
136,20,1.0,1.0
